In [ ]:
#Dezipper le dataset
!unzip dataset_augmente.zip

In [ ]:
#Installer tensorflow
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# --- 1. PARAMÈTRES ---
IMAGE_SIZE = (224, 224) # MobileNetV2 est optimisé pour cette taille
BATCH_SIZE = 16 # Ajuster selon la mémoire du GPU/CPU
DATASET_PATH = '/content/dataset_augmente'

# --- 2. CHARGEMENT ET PRÉPARATION DES DONNÉES ---

# Création des générateurs de données

train_dataset = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2, # 20% des données pour la validation
    subset="training",
    seed=123, # Pour la reproductibilité
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_dataset.class_names
print("Classes trouvées :", class_names)
NUM_CLASSES = len(class_names) # Devrait être 3

# Optimisation des performances de chargement
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

# --- 3. CONSTRUCTION DU MODÈLE AVEC TRANSFER LEARNING ---

# Chargement de MobileNetV2 pré-entraîné sur ImageNet, sans sa couche de classification finale
base_model = MobileNetV2(
    input_shape=IMAGE_SIZE + (3,),
    include_top=False, # Très important : on enlève la tête du classifieur original
    weights='imagenet'
)

# Geler les poids du modèle de base. On ne veut pas les ré-entraîner pour l'instant.
base_model.trainable = False

# Ajouter nos propres couches de classification par-dessus
# On prend la sortie du modèle de base.
x = base_model.output
# on la "globalise" pour la transformer en un simple vecteur
x = GlobalAveragePooling2D()(x)
# on ajoute une couche de Dropout pour éviter le sur-apprentissage (très important sur les petits datasets)
x = Dropout(0.3)(x)
# enfin, notre couche de prédiction finale avec 3 neurones (un par classe) et une activation softmax
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Créer le modèle final
model = Model(inputs=base_model.input, outputs=predictions)

# --- 4. COMPILATION DU MODÈLE ---
# On spécifie comment le modèle doit apprendre
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy', # Adapté pour les labels entiers fournis par image_dataset_from_directory
    metrics=['accuracy']
)

# Afficher un résumé de l'architecture
model.summary()

# --- 5. ENTRAÎNEMENT DU MODÈLE ---
print("\nDébut de l'entraînement...")
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=20 # On peut commencer avec 20 et augmenter si nécessaire
)

print("\nEntraînement terminé !")

# Sauvegarder le modèle pour une utilisation future
model.save('mon_modele_dechets.h5')

Found 234 files belonging to 3 classes.
Using 188 files for training.
Found 234 files belonging to 3 classes.
Using 46 files for validation.
Classes trouvées : ['Dangereux', 'Menager', 'Recyclable']
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,261,827 (8.63 MB)

 Trainable params: 3,843 (15.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)


Début de l'entraînement...
Epoch 1/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 936ms/step - accuracy: 0.3339 - loss: 1.5199 - val_accuracy: 0.3478 - val_loss: 1.1813
Epoch 2/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.4841 - loss: 1.1318 - val_accuracy: 0.6522 - val_loss: 0.9006
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.4468 - loss: 1.0200 - val_accuracy: 0.6522 - val_loss: 0.8501
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5815 - loss: 0.8909 - val_accuracy: 0.7174 - val_loss: 0.7663
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6768 - loss: 0.7075 - val_accuracy: 0.6957 - val_loss: 0.7086
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.6668 - loss: 0.6835 - val_accuracy: 0.7174 - val_loss: 0.7066
Epoch 7/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7610 - loss: 0.6071 - val_accuracy: 0.6957 - val_loss: 0.6797
Epoch 8/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.7842 - loss: 0.5

In [ ]:
model.save('mon_modele_dechets.h5')

# test

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import os

# --- 1. PARAMÈTRES ---
# Chemin vers le modèle sauvegardé
MODEL_PATH = '/content/mon_modele_dechets.h5'
# Chemin vers le dossier contenant les images à tester
IMAGES_FOLDER = '/content/images_a_tester'
# Taille des images attendue par le modèle (doit être la même que pour l'entraînement)
IMAGE_SIZE = (224, 224)

# IMPORTANT : Les noms des classes doivent être dans le même ordre que lors de l'entraînement.
# Keras les ordonne alphabétiquement par défaut. Vérifiez l'ordre dans vos dossiers.
CLASS_NAMES = ['Dangereux', 'Menager', 'Recyclable'] # A adapter si l'ordre est différent !

# --- 2. CHARGEMENT DU MODÈLE ---
print(f"Chargement du modèle depuis : {MODEL_PATH}")
try:
    model = tf.keras.models.load_model(MODEL_PATH)
    print("Modèle chargé avec succès.")
except Exception as e:
    print(f"Erreur lors du chargement du modèle : {e}")
    exit()

# --- 3. FONCTION DE PRÉDICTION POUR UNE SEULE IMAGE ---
def predict_image(img_path):
    """Charge, prépare une image et retourne la prédiction du modèle."""

    # Charger l'image depuis le chemin du fichier
    img = image.load_img(img_path, target_size=IMAGE_SIZE)

    # Convertir l'image en un tableau NumPy
    img_array = image.img_to_array(img)

    # Ajouter une dimension pour créer un "batch" d'une seule image
    # La forme passe de (224, 224, 3) à (1, 224, 224, 3)
    img_batch = np.expand_dims(img_array, axis=0)

    # **Pro Tip**: Si vous utilisez un modèle comme MobileNetV2, il est préférable
    # d'utiliser sa propre fonction de pré-traitement pour normaliser les pixels.
    # from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
    # img_batch = preprocess_input(img_batch)

    # Faire la prédiction
    predictions = model.predict(img_batch)

    # 'predictions' est un tableau de probabilités, ex: [[0.05, 0.92, 0.03]]
    # On trouve l'index de la probabilité la plus élevée
    predicted_index = np.argmax(predictions[0])

    # On récupère le nom de la classe correspondante
    predicted_class = CLASS_NAMES[predicted_index]

    # On récupère le score de confiance
    confidence = np.max(predictions[0]) * 100

    return predicted_class, confidence

# --- 4. BOUCLE DE TEST SUR TOUTES LES IMAGES DU DOSSIER ---
if not os.path.exists(IMAGES_FOLDER):
    print(f"Le dossier de test '{IMAGES_FOLDER}' n'existe pas.")
else:
    print("\n--- Début des prédictions ---")
    for filename in os.listdir(IMAGES_FOLDER):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(IMAGES_FOLDER, filename)

            predicted_class, confidence = predict_image(image_path)

            print(f"Image: {filename:20s} | Prédiction: {predicted_class:12s} | Confiance: {confidence:.2f}%")

Chargement du modèle depuis : /content/mon_modele_dechets.h5


Modèle chargé avec succès.

--- Début des prédictions ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Image: photo_4_aug_std_0.jpg | Prédiction: Menager      | Confiance: 97.07%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Image: photo_0_aug_std_1.jpg | Prédiction: Menager      | Confiance: 91.31%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Image: photo_2-removebg-preview_aug_bg_0.jpg | Prédiction: Dangereux    | Confiance: 42.72%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Image: photo_2_aug_std_3.jpg | Prédiction: Menager      | Confiance: 47.85%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Image: photo_4_aug_std_19.jpg | Prédiction: Menager      | Confiance: 95.05%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Image: photo_0_aug_std_17.jpg | Prédiction: Menager      | Confiance: 65.33%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Image: photo_0_aug_std_14.jpg | Prédiction: Menager      | Confiance: 95.44%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Image: photo_2-removebg-preview_aug_bg_2.jpg | Prédiction: Menager      | Confiance: 37.1